Lucía García-Duarte Sáenz
# GCN-LSTM Model

- Architecture: 
  1. User defined number of graph convolutional layers (Reference: Kipf & Welling (ICLR 2017)).
  2. User defined number of LSTM layers. The TGCN uses GRU instead of LSTM. In practice there are not any remarkable differences between the two types of layers. We use LSTM as they are more frequently used.
  3. A Dropout and a Dense layer as they experimentally showed improvement in performance and managing over-fitting.

In [ ]:
import os
import sys
import urllib.request

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

import stellargraph as sg
from stellargraph.layer import GCN_LSTM

## Load the data
- An NxN adjacency matrix, which describes the distance relationship between the N stations,
- An NxT feature matrix, which describes the ($T_1, \cdots, T_t$) temperature records over t timesteps for the N stations.

Temperature is recorded every hour over 37 stations for 26 years

In [ ]:
adj_mat = pd.read_csv('adj_matrix_1000.csv')
adj_mat = adj_mat.drop(columns=['Unnamed: 0'])
adj_mat = adj_mat.values

In [ ]:
feat_mat = pd.read_csv('feat_matrix.csv')
feat_mat = feat_mat.drop(columns=['Unnamed: 0'])

In [ ]:
num_nodes, time_len = feat_mat.shape
print("No. of stations:", num_nodes, "\nNo. of timesteps:", time_len)

## Prepare the data

Define functions to split the data into training, validation and test, to normalize the data, and to prepare the data to be fed into the neural network

In [ ]:
def train_val_test_split(data, valid_portion, test_portion):
    
    time_len = data.shape[1]
    train_portion = 1 - valid_portion - test_portion
    train_size = int(time_len * train_portion)
    valid_size = int(time_len * valid_portion)
    
    train_data = np.array(data.iloc[:, :train_size])
    valid_data = np.array(data.iloc[:, train_size:train_size+valid_size])
    test_data = np.array(data.iloc[:, train_size+valid_size:])
    
    return train_data, valid_data, test_data


def scale_data(train_data, valid_data, test_data):
    
    max_speed = train_data.max()
    min_speed = train_data.min()
    
    train_scaled = (train_data - min_speed) / (max_speed - min_speed)
    valid_scaled = (valid_data - min_speed) / (max_speed - min_speed)
    test_scaled = (test_data - min_speed) / (max_speed - min_speed)
    
    return train_scaled, valid_scaled, test_scaled


def sequence_data_preparation(seq_len, pre_len, train_data, valid_data, test_data):
    
    trainX, trainY, validX, validY, testX, testY = [], [], [], [], [], []

    for i in range(train_data.shape[1] - int(seq_len + pre_len - 1)):
        a = train_data[:, i : i + seq_len + pre_len]
        trainX.append(a[:, :seq_len])
        trainY.append(a[:, -1])
        
    for i in range(valid_data.shape[1] - int(seq_len + pre_len - 1)):
        b = valid_data[:, i : i + seq_len + pre_len]
        validX.append(b[:, :seq_len])
        validY.append(b[:, -1])
        
    for i in range(test_data.shape[1] - int(seq_len + pre_len - 1)):
        c = test_data[:, i : i + seq_len + pre_len]
        testX.append(c[:, :seq_len])
        testY.append(c[:, -1])

    trainX = np.array(trainX)
    trainY = np.array(trainY)
    validX = np.array(validX)
    validY = np.array(validY)
    testX = np.array(testX)
    testY = np.array(testY)

    return trainX, trainY, validX, validY, testX, testY

### Train-test split

In [ ]:
test_rate = 0.3
valid_rate = 0.1
train_data, valid_data, test_data = train_val_test_split(feat_mat, valid_rate, test_rate)
print("Train data: ", train_data.shape)
print("Valid data: ", valid_data.shape)
print("Test data: ", test_data.shape)
feat_mat.shape[1] - train_data.shape[1] - valid_data.shape[1] - test_data.shape[1] # check dimensions

### Scaling

In [ ]:
train_scaled, valid_scaled, test_scaled = scale_data(train_data, valid_data, test_data)
#del feat_mat # to save memory for training

### Data preparation and reshaping
- Each training observation are `seq_len` historical temperatures: the size of the past window of information
- Each training prediction is the temperature X hours later (`pre_len`): how far in the future does the model need to learn to predict

In [ ]:
seq_len = 24    # change by 24, 24*7, 24*7*2, 24*7*3 (h)
pre_len = 1     # change by 1, 2, 3 (h)

trainX, trainY, validX, validY, testX, testY = sequence_data_preparation(
    seq_len, pre_len, train_scaled, valid_scaled, test_scaled
)

print(trainX.shape)
print(trainY.shape)
print(validX.shape)
print(validY.shape)
print(testX.shape)
print(testY.shape)

## Prepare the model

In [ ]:
gcn_lstm = GCN_LSTM(
    seq_len=seq_len,
    adj=adj_mat,
    gc_layer_sizes=[16, 8],
    gc_activations=["relu", "relu"],
    lstm_layer_sizes=[16, 16],           # change by 16, 32, 64, 128, 256
    lstm_activations=["tanh", "tanh"],
    dropout = 0.2
)

x_input, x_output = gcn_lstm.in_out_tensors()
model = Model(inputs=x_input, outputs=x_output)
model.compile(optimizer="adam", loss="mae", metrics=["mse"])

#from keras.callbacks import ModelCheckpoint
#mc = ModelCheckpoint('models_GCN-LSTM/24_1_32_16/best_model.h5', monitor='val_loss', mode='min', verbose=0, save_best_only=True)

In [ ]:
model.summary()

### Train

In [ ]:
import datetime
start = datetime.datetime.now()
history = model.fit(
    trainX,
    trainY,
    epochs=100,
    batch_size=32,
    shuffle=True,
    #verbose=0,
    validation_data=(validX, validY)#, callbacks = [mc]
)
end = datetime.datetime.now()

In [ ]:
print(
    "Train loss: ",
    history.history["loss"][-1],
    "\nValid loss:",
    history.history["val_loss"][-1],
    "\nElapsed time: ",
    end - start
)

In [ ]:
sg.utils.plot_history(history)

## Evaluate

In [ ]:
##If loading best model
#from tensorflow import keras
#model=keras.models.load_model("models_GCN-LSTM/24_1_32_16/best_model.h5")
#model.summary()

In [ ]:
ythat = model.predict(trainX)
yhat = model.predict(testX)
yht = model.predict(validX)

In [ ]:
## Rescale values
max_speed = train_data.max()
min_speed = train_data.min()

## actual train and test values
train_rescref = np.array(trainY * max_speed)
test_rescref = np.array(testY * max_speed)
valid_rescref = np.array(validY * max_speed)


In [ ]:
train_rescpred = np.array((ythat) * max_speed)
test_rescpred = np.array((yhat) * max_speed)
valid_rescpred = np.array((yht) * max_speed)

In [ ]:
## Performance measures

from sklearn.metrics import r2_score, explained_variance_score#, mean_absolute_percentage_error
from sklearn.metrics.regression import _check_reg_targets, check_consistent_length

def mean_absolute_percentage_error(y_true, y_pred,
                                   sample_weight=None,
                                   multioutput='uniform_average'):

    y_type, y_true, y_pred, multioutput = _check_reg_targets(
        y_true, y_pred, multioutput)
    check_consistent_length(y_true, y_pred, sample_weight)
    epsilon = np.finfo(np.float64).eps
    mape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), epsilon)
    output_errors = np.average(mape,
                               weights=sample_weight, axis=0)
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            return output_errors
        elif multioutput == 'uniform_average':
            # pass None as weights to np.average: uniform mean
            multioutput = None

    return np.average(output_errors, weights=multioutput)

On the training set

In [ ]:
# Naïve
trainnpred = np.array(trainX)[
    :, :, -1
]  
trainnpredc = (trainnpred) * max_speed

# Metrics
seg_nmael = [] #naïve mae
seg_nmsel = [] #naïve mse
seg_nmape = [] #naïve mape
seg_nacc = []  #naïve accuracy
seg_nR2 = []   #naïve coeff of determination
seg_nvar = []  #naïve coeff of variance score

seg_mael = []  #mae
seg_masel = [] #mase
seg_msel = []  #mse
seg_mape = []  #mape
seg_acc = []   #accuracy
seg_R2 = []    #coeff of determination
seg_var = []   #coeff of variance score

kk=trainX.shape[1]
for j in range(kk):
    
    ## NAIVE
    # Mean Square Error 
    seg_nmsel.append(np.mean(np.square(train_rescref.T[j] - trainnpredc.T[j]))) 
        
    # Mean Absolute Error 
    seg_nmael.append(np.mean(np.abs(train_rescref.T[j] - trainnpredc.T[j])))  

    # Mean Absolute Percentage Error
    seg_nmape.append(mean_absolute_percentage_error(train_rescref.T[j],trainnpredc.T[j]))
    
    # Accuracy
    seg_nacc.append(1-np.linalg.norm(train_rescref.T[j]-trainnpredc.T[j])/np.linalg.norm(train_rescref.T[j]))
    
    # Coefficient of determination
    seg_nR2.append(r2_score(train_rescref.T[j],trainnpredc.T[j]))
    
    # Explained variance score
    seg_nvar.append(explained_variance_score(train_rescref.T[j],trainnpredc.T[j]))
    
    
    ## NN
    # Mean Absolute Error 
    seg_mael.append(np.mean(np.abs(train_rescref.T[j] - train_rescpred.T[j])))  
    
    # Mean Square Error 
    seg_msel.append(np.mean(np.square(train_rescref.T[j] - train_rescpred.T[j]))) 
       
    # Mean Absolute Percentage Error
    seg_mape.append(mean_absolute_percentage_error(train_rescref.T[j],train_rescpred.T[j]))
    
    # Accuracy
    seg_acc.append(1-np.linalg.norm(train_rescref.T[j]-train_rescpred.T[j])/np.linalg.norm(train_rescref.T[j]))
    
    # Coefficient of determination
    seg_R2.append(r2_score(train_rescref.T[j],train_rescpred.T[j]))
    
    # Explained variance score
    seg_var.append(explained_variance_score(train_rescref.T[j],train_rescpred.T[j]))
    
    if seg_nmael[-1] != 0:
        seg_masel.append(
            seg_mael[-1] / seg_nmael[-1]
        )  # Ratio of the two: Mean Absolute Scaled Error
    else:
        seg_masel.append(np.NaN)

print("Total (ave) MAE for NN: " + str(np.mean(np.array(seg_mael))))
print("Total (ave) RMSE for NN: " + str(np.mean(np.sqrt(np.array(seg_msel)))))
print("Total (ave) MAPE for NN: " + str(np.mean(np.array(seg_mape))))
print("Total (ave) Accuracy for NN: " + str(np.mean(np.array(seg_acc))))
print("Total (ave) R2 for NN: " + str(np.mean(np.array(seg_R2))))
print("Total (ave) Var for NN: " + str(np.mean(np.array(seg_var))))
print("Total (ave) MASE for per-segment NN/naive MAE: " + str(np.nanmean(np.array(seg_masel))))
print("...note that MASE<1 (for a given segment) means that the NN prediction is better than the naive prediction.")

print("\nTotal (ave) MAE for naive prediction: " + str(np.mean(np.array(seg_nmael))))
print("Total (ave) RMSE for naive prediction: " + str(np.mean(np.sqrt(np.array(seg_nmsel)))))
print("Total (ave) MAPE for naive prediction: " + str(np.mean(np.array(seg_nmape))))
print("Total (ave) Accuracy for naive prediction: " + str(np.mean(np.array(seg_nacc))))
print("Total (ave) R2 for naive prediction: " + str(np.mean(np.array(seg_nR2))))
print("Total (ave) Var for naive prediction: " + str(np.mean(np.array(seg_nvar))))

On the validation set

In [ ]:
# Naïve
validnpred = np.array(validX)[
    :, :, -1
] 
validnpredc = (validnpred) * max_speed

# Metrics
seg_nmael = [] #naïve mae
seg_nmsel = [] #naïve mse
seg_nmape = [] #naïve mape
seg_nacc = []  #naïve accuracy
seg_nR2 = []   #naïve coeff of determination
seg_nvar = []  #naïve coeff of variance score

seg_mael = []  #mae
seg_masel = [] #mase
seg_msel = []  #mse
seg_mape = []  #mape
seg_acc = []   #accuracy
seg_R2 = []    #coeff of determination
seg_var = []   #coeff of variance score

kk=validX.shape[1]
for j in range(kk):

    ## NAIVE
    # Mean Square Error
    seg_nmsel.append(np.mean(np.square(valid_rescref.T[j] - validnpredc.T[j])))

    # Mean Absolute Error
    seg_nmael.append(np.mean(np.abs(valid_rescref.T[j] - validnpredc.T[j])))

    # Mean Absolute Percentage Error
    seg_nmape.append(mean_absolute_percentage_error(valid_rescref.T[j], validnpredc.T[j]))

    # Accuracy
    seg_nacc.append(1 - np.linalg.norm(valid_rescref.T[j] - validnpredc.T[j]) / np.linalg.norm(valid_rescref.T[j]))

    # Coefficient of determination
    seg_nR2.append(r2_score(valid_rescref.T[j], validnpredc.T[j]))

    # Explained variance score
    seg_nvar.append(explained_variance_score(valid_rescref.T[j], validnpredc.T[j]))

    
    ## NN
    # Mean Absolute Error
    seg_mael.append(np.mean(np.abs(valid_rescref.T[j] - valid_rescpred.T[j])))

    # Mean Square Error
    seg_msel.append(np.mean(np.square(valid_rescref.T[j] - valid_rescpred.T[j])))

    # Mean Absolute Percentage Error
    seg_mape.append(mean_absolute_percentage_error(valid_rescref.T[j], valid_rescpred.T[j]))

    # Accuracy
    seg_acc.append(1 - np.linalg.norm(valid_rescref.T[j] - valid_rescpred.T[j]) / np.linalg.norm(valid_rescref.T[j]))

    # Coefficient of determination
    seg_R2.append(r2_score(valid_rescref.T[j], valid_rescpred.T[j]))

    # Explained variance score
    seg_var.append(explained_variance_score(valid_rescref.T[j], valid_rescpred.T[j]))

    if seg_nmael[-1] != 0:
        seg_masel.append(seg_mael[-1] / seg_nmael[-1])  # Ratio of the two: Mean Absolute Scaled Error
    else:
        seg_masel.append(np.NaN)

print("Total (ave) MAE for NN: " + str(np.mean(np.array(seg_mael))))
print("Total (ave) RMSE for NN: " + str(np.mean(np.sqrt(np.array(seg_msel)))))
print("Total (ave) MAPE for NN: " + str(np.mean(np.array(seg_mape))))
print("Total (ave) Accuracy for NN: " + str(np.mean(np.array(seg_acc))))
print("Total (ave) R2 for NN: " + str(np.mean(np.array(seg_R2))))
print("Total (ave) Var for NN: " + str(np.mean(np.array(seg_var))))
print("Total (ave) MASE for per-segment NN/naive MAE: " + str(np.nanmean(np.array(seg_masel))))
print("...note that MASE<1 (for a given segment) means that the NN prediction is better than the naive prediction.")

print("\nTotal (ave) MAE for naive prediction: " + str(np.mean(np.array(seg_nmael))))
print("Total (ave) RMSE for naive prediction: " + str(np.mean(np.sqrt(np.array(seg_nmsel)))))
print("Total (ave) MAPE for naive prediction: " + str(np.mean(np.array(seg_nmape))))
print("Total (ave) Accuracy for naive prediction: " + str(np.mean(np.array(seg_nacc))))
print("Total (ave) R2 for naive prediction: " + str(np.mean(np.array(seg_nR2))))
print("Total (ave) Var for naive prediction: " + str(np.mean(np.array(seg_nvar))))

On the test set

In [ ]:
# Naïve
testnpred = np.array(testX)[
    :, :, -1
] 
testnpredc = (testnpred) * max_speed

# Metrics
seg_nmael = [] #naïve mae
seg_nmsel = [] #naïve mse
seg_nmape = [] #naïve mape
seg_nacc = []  #naïve accuracy
seg_nR2 = []   #naïve coeff of determination
seg_nvar = []  #naïve coeff of variance score

seg_mael = []  #mae
seg_masel = [] #mase
seg_msel = []  #mse
seg_mape = []  #mape
seg_acc = []   #accuracy
seg_R2 = []    #coeff of determination
seg_var = []   #coeff of variance score

kk=testX.shape[1]
for j in range(kk):

    ## NAIVE
    # Mean Square Error
    seg_nmsel.append(np.mean(np.square(test_rescref.T[j] - testnpredc.T[j])))

    # Mean Absolute Error
    seg_nmael.append(np.mean(np.abs(test_rescref.T[j] - testnpredc.T[j])))

    # Mean Absolute Percentage Error
    seg_nmape.append(mean_absolute_percentage_error(test_rescref.T[j], testnpredc.T[j]))

    # Accuracy
    seg_nacc.append(1 - np.linalg.norm(test_rescref.T[j] - testnpredc.T[j]) / np.linalg.norm(test_rescref.T[j]))

    # Coefficient of determination
    seg_nR2.append(r2_score(test_rescref.T[j], testnpredc.T[j]))

    # Explained variance score
    seg_nvar.append(explained_variance_score(test_rescref.T[j], testnpredc.T[j]))

    
    ## NN
    # Mean Absolute Error
    seg_mael.append(np.mean(np.abs(test_rescref.T[j] - test_rescpred.T[j])))

    # Mean Square Error
    seg_msel.append(np.mean(np.square(test_rescref.T[j] - test_rescpred.T[j])))

    # Mean Absolute Percentage Error
    seg_mape.append(mean_absolute_percentage_error(test_rescref.T[j], test_rescpred.T[j]))

    # Accuracy
    seg_acc.append(1 - np.linalg.norm(test_rescref.T[j] - test_rescpred.T[j]) / np.linalg.norm(test_rescref.T[j]))

    # Coefficient of determination
    seg_R2.append(r2_score(test_rescref.T[j], test_rescpred.T[j]))

    # Explained variance score
    seg_var.append(explained_variance_score(test_rescref.T[j], test_rescpred.T[j]))

    if seg_nmael[-1] != 0:
        seg_masel.append(seg_mael[-1] / seg_nmael[-1])  # Ratio of the two: Mean Absolute Scaled Error
    else:
        seg_masel.append(np.NaN)

print("Total (ave) MAE for NN: " + str(np.mean(np.array(seg_mael))))
print("Total (ave) RMSE for NN: " + str(np.mean(np.sqrt(np.array(seg_msel)))))
print("Total (ave) MAPE for NN: " + str(np.mean(np.array(seg_mape))))
print("Total (ave) Accuracy for NN: " + str(np.mean(np.array(seg_acc))))
print("Total (ave) R2 for NN: " + str(np.mean(np.array(seg_R2))))
print("Total (ave) Var for NN: " + str(np.mean(np.array(seg_var))))
print("Total (ave) MASE for per-segment NN/naive MAE: " + str(np.nanmean(np.array(seg_masel))))
print("...note that MASE<1 (for a given segment) means that the NN prediction is better than the naive prediction.")

print("\nTotal (ave) MAE for naive prediction: " + str(np.mean(np.array(seg_nmael))))
print("Total (ave) RMSE for naive prediction: " + str(np.mean(np.sqrt(np.array(seg_nmsel)))))
print("Total (ave) MAPE for naive prediction: " + str(np.mean(np.array(seg_nmape))))
print("Total (ave) Accuracy for naive prediction: " + str(np.mean(np.array(seg_nacc))))
print("Total (ave) R2 for naive prediction: " + str(np.mean(np.array(seg_nR2))))
print("Total (ave) Var for naive prediction: " + str(np.mean(np.array(seg_nvar))))

In [ ]:
mypath = 'metrics/GCN_LSTM_24_1_32_16/'    #seq_len _ pre_len _ batch_size _ hidden_units

a_file = open(mypath+"MAE.txt", "w")
for row in np.matrix(seg_mael):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"RMSE.txt", "w")
for row in np.matrix(seg_msel):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"MASE.txt", "w")
for row in np.matrix(seg_masel):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"ACC.txt", "w")
for row in np.matrix(seg_acc):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"R2.txt", "w")
for row in np.matrix(seg_R2):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"EVS.txt", "w")
for row in np.matrix(seg_var):
    np.savetxt(a_file, row)
a_file.close()

In [ ]:
##Result visualization
fig1 = plt.figure(figsize=(15, 8))
#    ax1 = fig1.add_subplot(1,1,1)
a_pred = test_rescpred[0:, 20]            # select one station (here 20)
a_true = test_rescref[0:, 20]             # select one station (here 20)
plt.plot(a_true, "tab:green", label="true",linewidth=0.5)
plt.plot(a_pred, "tab:blue", label="prediction",linewidth=0.5)
plt.xlabel("time")
plt.ylabel("speed")
plt.legend(loc="best", fontsize=10)
plt.show()

## Save results

In [ ]:
mypath = 'models_GCN-LSTM/24_1_32_16/'    #seq_len _ pre_len _ batch_size _ hidden_units
model.save(mypath + 'model.h5')

a_file = open(mypath+"train_rescref.txt", "w")
for row in np.matrix(train_rescref):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"train_rescpred.txt", "w")
for row in np.matrix(train_rescpred):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"valid_rescref.txt", "w")
for row in np.matrix(valid_rescref):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"valid_rescpred.txt", "w")
for row in np.matrix(valid_rescpred):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"test_rescref.txt", "w")
for row in np.matrix(test_rescref):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"test_rescpred.txt", "w")
for row in np.matrix(test_rescpred):
    np.savetxt(a_file, row)
a_file.close()